In [ ]:
import paltas

root_path = paltas.__path__[0][:-7]

# Understanding the Configuration File

__Author:__ Sebastian Wagner-Carena

__Goals:__ 

1. Understand the basic philosophy of the manada configuration file
2. Understand how to specify all of the required parameters in a configuration file
3. Understand how to implement the optional parameters in a configuration file.

### Table of Contents

1. [The Basics](#basics)
2. [Required Parameters](#required_params) 
    1. [Source](#source)
    2. [Main Deflector](#main_deflector)
    3. [Cosmology](#cosmology)
    4. [Detector and PSF](#detector)
    5. [Numerical Parameters](#num_params)
3. [Optional Parameters](#optional_params)
    1. [Subhalos](#subhalos)
    2. [Line-of-Sight Halos](#los)
    3. [Masks and Cuts](#mask_cuts)
    4. [Drizzle](#drizzle)
    5. [Cross-Object Parameters](#cross_object)
    

## The Basics <a class="anchor" id="basics"></a>

## Required Parameters <a class="anchor" id="required_params"></a>

### Source <a class="anchor" id="source"></a>

### Main Deflector <a class="anchor" id="main_deflector"></a>

### Cosmology <a class="anchor" id="cosmology"></a>

### Detector and PSF <a class="anchor" id="detector"></a>

### Numerical Parameters <a class="anchor" id="num_params"></a>

## Optional Parameters <a class="anchor" id="optional_params"></a>

### Subhalos <a class="anchor" id="subhalos"></a>

### Line-of-Sight Halos <a class="anchor" id="los"></a>

### Masks and Cuts <a class="anchor" id="mask_cuts"></a>

### Drizzle <a class="anchor" id="drizzle"></a>

### Cross-Object Parameters <a class="anchor" id="cross_object"></a>